In [1]:
import cv2
import numpy as np
import csv
import math

In [62]:
map_index = 3
file_name = f'perumdos-{map_index}.png'

map_image = cv2.imread(file_name, cv2.IMREAD_COLOR)
map_image = cv2.resize(map_image, (1920, 1080))
img = map_image.copy()

coordinates = []

with open(f'coordinates-{map_index}.csv', mode='r') as file:
  reader = csv.reader(file)
  next(reader)
  for row in reader:
    x, y = int(row[1]), int(row[2])
    coordinates.append((x, y))
    cv2.circle(img, (x, y), 5, (0, 0, 255), -1)

print(coordinates)

[]


In [63]:
def print_coordinates(event, x, y, flags, param):
	if event == cv2.EVENT_LBUTTONDOWN:
		coordinates.append((x, y))
		# print(f"[{x}, {y}],")
		cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
		cv2.imshow("Image", img)

cv2.imshow("Image", img)
cv2.setMouseCallback("Image", print_coordinates)

while True:
	key = cv2.waitKey(1) & 0xFF

	if key == ord('d'):
		if len(coordinates) > 0:
			coordinates.pop()
			img = map_image.copy()
			for coord in coordinates:
				cv2.circle(img, coord, 5, (0, 0, 255), -1)
			cv2.imshow("Image", img)

	elif key == 27:
		break

cv2.destroyAllWindows()

with open(f'coordinates-{map_index}.csv', mode='w', newline='') as file:
	writer = csv.writer(file)
	writer.writerow(['No', 'X', 'Y'])
	for i, coord in enumerate(coordinates):
		writer.writerow([i, coord[0], coord[1]])

print(coordinates)

[(709, 84), (765, 85), (817, 87), (845, 88), (903, 89), (931, 90), (968, 89), (1014, 98), (1049, 69), (1075, 90), (1109, 88), (1142, 88), (1189, 87), (1230, 92), (737, 151), (733, 190), (733, 236), (736, 270), (738, 305), (737, 332), (736, 361), (736, 398), (739, 442), (738, 471), (735, 504), (739, 527), (735, 566), (769, 566), (788, 579), (784, 540), (778, 504), (782, 468), (782, 447), (784, 396), (786, 358), (782, 331), (781, 304), (782, 268), (783, 242), (783, 212), (781, 184), (780, 145), (841, 153), (843, 186), (842, 212), (847, 241), (852, 278), (842, 301), (848, 328), (839, 362), (840, 389), (836, 424), (843, 442), (842, 479), (841, 500), (840, 522), (841, 560), (837, 579), (892, 562), (890, 536), (890, 505), (890, 475), (890, 446), (890, 423), (892, 384), (895, 359), (891, 331), (889, 303), (890, 264), (890, 211), (889, 182), (886, 153), (957, 146), (948, 200), (947, 244), (946, 273), (938, 310), (938, 327), (951, 393), (949, 424), (949, 448), (958, 484), (993, 529)]


In [20]:
# Fungsi untuk menghitung jarak Euclidean antara dua titik (x1, y1) dan (x2, y2)
def euclidean_distance(p1, p2):
  return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

In [50]:
# Map 1 Scale
# x1, y1 = 657, 156
# x2, y2 = 709, 147
# real_distance = 28.78

# Map 2 Scale
# x1, y1 = 829, 177
# x2, y2 = 834, 217
# real_distance = 17.9

# Map 3 Scale
# x1, y1 = 294, 218
# x2, y2 = 425, 219
# real_distance = 45.3

point1 = [
  (657, 156),
  (829, 177),
  (294, 218)
]

point2 = [
  (709, 147),
  (834, 217),
  (425, 219)
]

real_distance = [
  28.78,
  17.9,
  45.3
]

scale = real_distance[map_index-1] / euclidean_distance(point1[map_index-1], point2[map_index-1])
print(scale)

0.3457914519429337


In [51]:
from scipy.spatial import distance_matrix

distances = distance_matrix(coordinates, coordinates)
n = len(coordinates)

mst_edges = []
visited = [False] * n
edge_costs = [float('inf')] * n
parent = [-1] * n

edge_costs[0] = 0

for _ in range(n):
    min_cost = float('inf')
    u = -1
    for v in range(n):
        if not visited[v] and edge_costs[v] < min_cost:
            min_cost = edge_costs[v]
            u = v

    visited[u] = True

    if parent[u] != -1:
        mst_edges.append((parent[u], u))

    for v in range(n):
        if not visited[v] and distances[u][v] < edge_costs[v]:
            edge_costs[v] = distances[u][v]
            parent[v] = u

img_result_prim = cv2.imread(file_name, cv2.IMREAD_COLOR)
img_result_prim = cv2.resize(img_result_prim, (1920, 1080))

for index, coord in enumerate(coordinates):
    cv2.circle(img_result_prim, tuple(coord), 5, (0, 0, 255), -1)
    cv2.putText(img_result_prim, str(index + 1), tuple(coord), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

for edge in mst_edges:
    pt1 = tuple(coordinates[edge[0]])
    pt2 = tuple(coordinates[edge[1]])
    cv2.line(img_result_prim, pt1, pt2, (255, 0, 0), 2)

cv2.imshow("MST Prim Result", img_result_prim)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Edges in MST:")
total_weight = 0
for edge in mst_edges:
    u, v = edge
    weight = distances[u][v] * scale
    total_weight += weight
    print(f"{u} -> {v} dengan jarak {weight:.2f}")
print(f"Total jarak MST: {total_weight:.2f}")

ValueError: not enough values to unpack (expected 2, got 1)

In [27]:
# Kruskal's Algorithm
from scipy.spatial import distance_matrix

distances = distance_matrix(coordinates, coordinates)

edges = []
for i, coord1 in enumerate(coordinates):
  for j, coord2 in enumerate(coordinates):
    if i < j:
      dist = distances[i][j]
      edges.append((dist, i, j))

edges.sort()

parent = list(range(len(coordinates)))
rank = [0] * len(coordinates)

def find(x):
  if parent[x] != x:
    parent[x] = find(parent[x])
  return parent[x]

def union(x, y):
  rootX = find(x)
  rootY = find(y)

  if rootX != rootY:
    if rank[rootX] > rank[rootY]:
      parent[rootY] = rootX
    elif rank[rootX] < rank[rootY]:
      parent[rootX] = rootY
    else:
      parent[rootY] = rootX
      rank[rootX] += 1

mst_edges = []
total_weight = 0

for dist, u, v in edges:
  if find(u) != find(v):
    union(u, v)
    mst_edges.append((u, v, dist))
    total_weight += dist

print("Edges in MST:")
for edge in mst_edges:
  print(f"{edge[0]} -> {edge[1]} with distance {edge[2] * scale:.2f}")
print(f"Total distance of MST: {total_weight * scale:.2f}")

img_result_kruskal = cv2.imread(file_name, cv2.IMREAD_COLOR)
img_result_kruskal = cv2.resize(img_result_kruskal, (1920, 1080))

for index, node in enumerate(coordinates):
  cv2.circle(img_result_kruskal, node, 5, (0, 0, 255), -1)
  cv2.putText(img_result_kruskal, str(index+1), node, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

for edge in mst_edges:
  pt1 = coordinates[edge[0]]
  pt2 = coordinates[edge[1]]
  cv2.line(img_result_kruskal, pt1, pt2, (255, 0, 0), 2)

cv2.imshow("MST Kruskal Result", img_result_kruskal)
cv2.waitKey(0)
cv2.destroyAllWindows()

Edges in MST:
104 -> 105 with distance 6.66
88 -> 89 with distance 7.10
107 -> 108 with distance 7.32
112 -> 113 with distance 7.32
62 -> 63 with distance 7.44
0 -> 1 with distance 7.56
133 -> 136 with distance 8.16
64 -> 65 with distance 8.19
13 -> 14 with distance 8.43
59 -> 60 with distance 8.89
26 -> 27 with distance 8.99
129 -> 132 with distance 9.14
15 -> 17 with distance 9.15
124 -> 130 with distance 9.34
139 -> 140 with distance 9.37
136 -> 137 with distance 9.42
109 -> 110 with distance 9.44
2 -> 3 with distance 9.49
22 -> 23 with distance 9.57
8 -> 9 with distance 9.59
132 -> 134 with distance 9.61
23 -> 24 with distance 9.81
15 -> 16 with distance 9.93
19 -> 20 with distance 9.93
116 -> 117 with distance 9.98
113 -> 114 with distance 10.02
4 -> 5 with distance 10.13
125 -> 129 with distance 10.22
7 -> 8 with distance 10.25
6 -> 7 with distance 10.39
3 -> 4 with distance 10.45
5 -> 6 with distance 10.45
9 -> 10 with distance 10.55
108 -> 109 with distance 10.55
17 -> 18 with 

In [46]:
# Displaying Comparison

crop_size = [
  (500, 1400),
  (400, 1450),
]

image1 = img_result_prim.copy()
image2 = img_result_kruskal.copy()

height, width, _ = image1.shape

image1 = image1[0:height, crop_size[map_index-1][0]:crop_size[map_index-1][1]]
image2 = image2[0:height, crop_size[map_index-1][0]:crop_size[map_index-1][1]]

cv2.rectangle(image1, (0, 0), (image1.shape[1], 50), (0, 0, 0), -1)
cv2.rectangle(image2, (0, 0), (image2.shape[1], 50), (0, 0, 0), -1)

cv2.putText(image1, "Prim's", (int(image1.shape[1]*0.5) - 20, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
cv2.putText(image2, "Kruskal's", (int(image2.shape[1]*0.5) - 50, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

comparison = np.hstack((image1, image2))
cv2.line(comparison, (image1.shape[1], 0), (image1.shape[1], height), (0, 0, 0), 2)

height, width, _ = comparison.shape
new_height = int(height * 0.9)
new_width = int(width * 0.9)
comparison = cv2.resize(comparison, (new_width, new_height))

cv2.imshow("Comparison", comparison)
cv2.waitKey(0)
cv2.destroyAllWindows()